<a href="https://colab.research.google.com/github/Carlos-Oliver-O/Carlos-Oliver-O/blob/main/01_DataFramesBascis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DataFrames Basics

## Prerrequisites

Install Spark and Java in VM

In [1]:
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark 3.5.3
!wget /q https://dlcdn.apache.org/spark/spark-3.5.3/spark-3.5.3-bin-hadoop3.tgz

/q: Scheme missing.
--2024-12-03 17:29:25--  https://dlcdn.apache.org/spark/spark-3.5.3/spark-3.5.3-bin-hadoop3.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 400864419 (382M) [application/x-gzip]
Saving to: ‘spark-3.5.3-bin-hadoop3.tgz’

spark-3.5.3-bin-had 100%[===================>] 382.29M   184MB/s    in 2.1s    

2024-12-03 17:29:28 (184 MB/s) - ‘spark-3.5.3-bin-hadoop3.tgz’ saved [400864419/400864419]

FINISHED --2024-12-03 17:29:28--
Total wall clock time: 2.2s
Downloaded: 1 files, 382M in 2.1s (184 MB/s)


In [2]:
ls -l # check the .tgz is there

total 391476
drwxr-xr-x 1 root root      4096 Nov 25 19:13 sample_data/
-rw-r--r-- 1 root root 400864419 Sep  9 05:35 spark-3.5.3-bin-hadoop3.tgz


In [3]:
# unzip it
!tar xf spark-3.5.3-bin-hadoop3.tgz

In [4]:
!pip install -q findspark

In [5]:

!pip install py4j

# For maps
!pip install folium
!pip install plotly

Define the environment

In [6]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[*] pyspark-shell"

Start Spark Session

---

In [7]:
import findspark
findspark.init("spark-3.5.3-bin-hadoop3")# SPARK_HOME

from pyspark.sql import SparkSession

# create the session
spark = SparkSession \
        .builder \
        .appName("DataFrames Basics") \
        .master("local[*]") \
        .getOrCreate()

spark.version

In [8]:
spark

In [9]:
# For Pandas conversion optimization
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [10]:
# Import sql functions
from pyspark.sql.functions import *

Download datasets

In [11]:
!mkdir -p dataset
!wget -q https://raw.githubusercontent.com/paponsro/spark_edem_2324/master/dataset/cars.json -P /dataset
!wget -q https://raw.githubusercontent.com/paponsro/spark_edem_2324/master/dataset/movies.json -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/bank.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/vehicles.csv -P /dataset
!ls /dataset

bank.csv  cars.json  movies.json  vehicles.csv


In [12]:
ls -l /dataset

total 1784
-rw-r--r-- 1 root root  461474 Dec  3 17:30 bank.csv
-rw-r--r-- 1 root root   74910 Dec  3 17:30 cars.json
-rw-r--r-- 1 root root 1274347 Dec  3 17:30 movies.json
-rw-r--r-- 1 root root    4370 Dec  3 17:30 vehicles.csv


## Examples

In [13]:
from pyspark.sql.types import Row
from pyspark.sql.functions import *

In [14]:
bankText = spark.sparkContext.textFile("/dataset/bank.csv")

#we have to: remove first row (headers), map the rest, and create DF
bank = bankText.map(lambda lineaCsv: lineaCsv.split(";"))\
.filter(lambda s: s[0] != "\"age\"") \
.map(lambda row: Row(int(row[0]), row[1].replace("\"", ""), row[2].replace("\"", ""), row[3].replace("\"", ""), row[5].replace("\"", ""))) \
.toDF(["age", "job", "marital", "education", "balance"]) \
.withColumn("age", col("age").cast("int"))

bank.show(3)

+---+----------+-------+---------+-------+
|age|       job|marital|education|balance|
+---+----------+-------+---------+-------+
| 30|unemployed|married|  primary|   1787|
| 33|  services|married|secondary|   4789|
| 35|management| single| tertiary|   1350|
+---+----------+-------+---------+-------+
only showing top 3 rows



Read directly from JSON file to a DF

In [15]:
carsDF = spark.read.option("inferSchema", True).json("/dataset/cars.json") # inferSchema requires one extra pass over the data

# if None is set, it uses de default value (default = False) you can also pass the schema manually

Read directly from csv

In [16]:
bankDF = spark.read.option("header", "true").option("delimiter", ";").csv("/dataset/bank.csv")
bankDF.show(3)

+---+----------+-------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---+
|age|       job|marital|education|default|balance|housing|loan| contact|day|month|duration|campaign|pdays|previous|poutcome|  y|
+---+----------+-------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---+
| 30|unemployed|married|  primary|     no|   1787|     no|  no|cellular| 19|  oct|      79|       1|   -1|       0| unknown| no|
| 33|  services|married|secondary|     no|   4789|    yes| yes|cellular| 11|  may|     220|       1|  339|       4| failure| no|
| 35|management| single| tertiary|     no|   1350|    yes|  no|cellular| 16|  apr|     185|       1|  330|       1| failure| no|
+---+----------+-------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---+
only showing top 3 rows



Showing a DF and print schema

In [17]:
carsDF.show(2)
carsDF.printSchema()

+------------+---------+------------+----------+----------------+--------------------+------+-------------+----------+
|Acceleration|Cylinders|Displacement|Horsepower|Miles_per_Gallon|                Name|Origin|Weight_in_lbs|      Year|
+------------+---------+------------+----------+----------------+--------------------+------+-------------+----------+
|        12.0|        8|       307.0|       130|            18.0|chevrolet chevell...|   USA|         3504|1970-01-01|
|        11.5|        8|       350.0|       165|            15.0|   buick skylark 320|   USA|         3693|1970-01-01|
+------------+---------+------------+----------+----------------+--------------------+------+-------------+----------+
only showing top 2 rows

root
 |-- Acceleration: double (nullable = true)
 |-- Cylinders: long (nullable = true)
 |-- Displacement: double (nullable = true)
 |-- Horsepower: long (nullable = true)
 |-- Miles_per_Gallon: double (nullable = true)
 |-- Name: string (nullable = true)
 |-- 

Get Rows

In [18]:
carsDF.take(2)

[Row(Acceleration=12.0, Cylinders=8, Displacement=307.0, Horsepower=130, Miles_per_Gallon=18.0, Name='chevrolet chevelle malibu', Origin='USA', Weight_in_lbs=3504, Year='1970-01-01'),
 Row(Acceleration=11.5, Cylinders=8, Displacement=350.0, Horsepower=165, Miles_per_Gallon=15.0, Name='buick skylark 320', Origin='USA', Weight_in_lbs=3693, Year='1970-01-01')]

Count

In [19]:
carsDF.count()

406

Schema

In [20]:
# obtain a schema
carsSchema = carsDF.schema
print(type(carsSchema))
print(carsSchema)

<class 'pyspark.sql.types.StructType'>
StructType([StructField('Acceleration', DoubleType(), True), StructField('Cylinders', LongType(), True), StructField('Displacement', DoubleType(), True), StructField('Horsepower', LongType(), True), StructField('Miles_per_Gallon', DoubleType(), True), StructField('Name', StringType(), True), StructField('Origin', StringType(), True), StructField('Weight_in_lbs', LongType(), True), StructField('Year', StringType(), True)])


Custom Schemas

In [21]:
example = spark.sparkContext.parallelize([("chevrolet chevelle malibu",18,"1970-01-01","USA"),
    ("buick skylark 320",15,"1970-01-01","USA")])

In [22]:
exampleDF = spark.createDataFrame(example)
exampleDF.printSchema()

root
 |-- _1: string (nullable = true)
 |-- _2: long (nullable = true)
 |-- _3: string (nullable = true)
 |-- _4: string (nullable = true)



In [23]:
exampleDF.show()

+--------------------+---+----------+---+
|                  _1| _2|        _3| _4|
+--------------------+---+----------+---+
|chevrolet chevell...| 18|1970-01-01|USA|
|   buick skylark 320| 15|1970-01-01|USA|
+--------------------+---+----------+---+



With columns names

In [24]:
names = list(["name", "weight", "date", "country"])

In [25]:
example2DF = example.toDF(names)
example2DF.printSchema()

root
 |-- name: string (nullable = true)
 |-- weight: long (nullable = true)
 |-- date: string (nullable = true)
 |-- country: string (nullable = true)



In [26]:
example2DF.show()

+--------------------+------+----------+-------+
|                name|weight|      date|country|
+--------------------+------+----------+-------+
|chevrolet chevell...|    18|1970-01-01|    USA|
|   buick skylark 320|    15|1970-01-01|    USA|
+--------------------+------+----------+-------+



In [27]:
# importing sql types
from pyspark.sql.types import *

In [28]:
# custom schema
customSchema = StructType([ \
    StructField('name', StringType(), True), \
    StructField('weight', StringType(), True), \
    StructField('date', StringType(), True), \
    StructField('country', StringType(), True)])

In [29]:
example3DF = spark.createDataFrame(example, customSchema)
example3DF.printSchema()

root
 |-- name: string (nullable = true)
 |-- weight: string (nullable = true)
 |-- date: string (nullable = true)
 |-- country: string (nullable = true)



In [30]:
example3DF.show(2, False)

+-------------------------+------+----------+-------+
|name                     |weight|date      |country|
+-------------------------+------+----------+-------+
|chevrolet chevelle malibu|18    |1970-01-01|USA    |
|buick skylark 320        |15    |1970-01-01|USA    |
+-------------------------+------+----------+-------+



In [31]:
# we can also specify schema with DDL (Data Definition Language)
customSchema2 = "`name` STRING NOT NULL, `weight` INT, `date` STRING, `country` STRING"

In [32]:
example4DF = spark.createDataFrame(example, customSchema2)
example4DF.printSchema()

root
 |-- name: string (nullable = false)
 |-- weight: integer (nullable = true)
 |-- date: string (nullable = true)
 |-- country: string (nullable = true)



In [33]:
print(type(example2DF.collect()[0]["weight"]))
print(type(example3DF.collect()[0]["weight"]))

<class 'int'>
<class 'str'>


## Exercises
1) Create a manual DF describing smartphones
  - maker
  - model
  - screen dimension
  - camera megapixels
  
2) Read another file from the dataset/ folder, e.g. movies.json
  - print its schema
  - count the number of rows, call count()

3) Take a look to vehicles.csv. Read the file to a DF but this time with your own schema

Exercise 1

In [57]:
example = spark.sparkContext.parallelize([("Iphone",14,"10x10","64mpx"),
    ("buick skylark 320",15,"1970-01-01","USA"), ("Iphone",14,"10x10","64mpx")])

In [58]:
names = list(["maker", "model", "screen dimension", "camera megapixels"])
ejer1_DF = example.toDF(names)
ejer1_DF.printSchema()

root
 |-- maker: string (nullable = true)
 |-- model: long (nullable = true)
 |-- screen dimension: string (nullable = true)
 |-- camera megapixels: string (nullable = true)



In [60]:
ejer1_DF.show(3, False)


+-----------------+-----+----------------+-----------------+
|maker            |model|screen dimension|camera megapixels|
+-----------------+-----+----------------+-----------------+
|Iphone           |14   |10x10           |64mpx            |
|buick skylark 320|15   |1970-01-01      |USA              |
|Iphone           |14   |10x10           |64mpx            |
+-----------------+-----+----------------+-----------------+



In [61]:
ejer1_DF.count()

3

Exercise 2

In [62]:
# Exercise 2
moviesDF = spark.read.option("inferSchema", True).json("/dataset/movies.json")
moviesDF.printSchema()
print(f"Number of rows: {moviesDF.count()}")

root
 |-- Creative_Type: string (nullable = true)
 |-- Director: string (nullable = true)
 |-- Distributor: string (nullable = true)
 |-- IMDB_Rating: double (nullable = true)
 |-- IMDB_Votes: long (nullable = true)
 |-- MPAA_Rating: string (nullable = true)
 |-- Major_Genre: string (nullable = true)
 |-- Production_Budget: long (nullable = true)
 |-- Release_Date: string (nullable = true)
 |-- Rotten_Tomatoes_Rating: long (nullable = true)
 |-- Running_Time_min: long (nullable = true)
 |-- Source: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- US_DVD_Sales: long (nullable = true)
 |-- US_Gross: long (nullable = true)
 |-- Worldwide_Gross: long (nullable = true)

Number of rows: 3201


Exercise 3

In [63]:
vehicleSchema = StructType([
    StructField("Car", StringType(), True),
    StructField("MPG", DoubleType(), True),
    StructField("Cylinders", IntegerType(), True),
    StructField("Displacement", DoubleType(), True),
    StructField("Horsepower", DoubleType(), True),
    StructField("Weight", IntegerType(), True),
    StructField("Acceleration", DoubleType(), True),
    StructField("Model", IntegerType(), True),
    StructField("Origin", StringType(), True)
])

vehiclesDF = spark.read.csv("/dataset/vehicles.csv", header=True, schema=vehicleSchema)

vehiclesDF.printSchema()
vehiclesDF.show(5)

root
 |-- Car: string (nullable = true)
 |-- MPG: double (nullable = true)
 |-- Cylinders: integer (nullable = true)
 |-- Displacement: double (nullable = true)
 |-- Horsepower: double (nullable = true)
 |-- Weight: integer (nullable = true)
 |-- Acceleration: double (nullable = true)
 |-- Model: integer (nullable = true)
 |-- Origin: string (nullable = true)

+------------------+----+---------+------------+----------+------+------------+-----+------+
|               Car| MPG|Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|
+------------------+----+---------+------------+----------+------+------------+-----+------+
|      Sand Crawler|NULL|     NULL|    150000.0|      36.8|    30|        46.0|   30| 50000|
|    T-16 skyhopper|NULL|     NULL|     14500.0|      10.4|  1200|         1.0|    1|    50|
|  X-34 landspeeder|NULL|     NULL|     10550.0|       3.4|   250|         1.0|    1|     5|
|TIE/LN starfighter|NULL|     NULL|        NULL|       6.4|  1200|         1.0|